# HW -2

rc47878

# Imports and Helper Functions

In [3]:
import numpy as np
import gurobi as gb
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize

GUROBI_TIMEOUT = 5 # in second

%matplotlib inline

In [4]:
# Helper Functions

class Gurobi_Model():
    '''
        Class Method I made to help me encapsulate the whole Gurobi model creation, 
        Finding Slack Variable and perform sensitivity analysis
    '''
    
    def __init__(self, obj_linear, cons, sense, b, variable_types = 'C',
                lower_bound = None, upper_bound = None,
                optimisation_type = gb.GRB.MAXIMIZE, obj_quadratic = None) -> None:
        '''
            Initialising the Model
        '''
        if (obj_quadratic is None) and (obj_linear is None):
            raise Exception("No objective defined")

        constraints, variables = cons.shape

        ## Optimisation Model at work (Maximise)
        try:
            self.model = gb.Model()
            self.model_X = self.model.addMVar(variables,
                                        vtype=variable_types,
                                        lb= lower_bound, ub= upper_bound)

            self.model_constraints = self.model.addMConstrs(cons, self.model_X, sense, b)
            self.model.setMObjective(obj_quadratic, obj_linear, 0, sense=optimisation_type)
            self.model.Params.OutputFlag = 0
            self.model.Params.TimeLimit = GUROBI_TIMEOUT

            self.model.optimize()

        except Exception:
            print("Error in optimising")
            raise Exception
        
        ## Assigning stuff for other use cases
        try:
            self.obj_q = obj_quadratic
            self.obj = obj_linear
            self.constraint = cons
            self.sense = sense
            self.b = b
        except Exception:
            print("Error in Storing equations")
            raise Exception
        return

    @property
    def optimal_obj(self): return self.model.objVal

    @property
    def optimal_x(self): return self.model_X.x

    @property
    def output(self):
        return {'objVal': self.optimal_obj,
                'x': self.optimal_x}

    def print_equations(self)->None:
        '''
            Print Objective Functions and Constraint Equations
        '''
        char = "a"
        print("Optimise System of equations:")
        for item in self.obj:
            print(str(item)+char,end=" + ")
            char = chr(ord(char) + 1)
        print("\b\b")
        print("Subject to:")#, end=" ")
        for i in range(self.constraint.shape[0]):
            char = "a"
            print("\t"+str(i)+")",end=" ")
            for j in range(self.constraint.shape[1]):
                print(str(self.constraint[i,j])+char,end=" + ")
                char = chr(ord(char) + 1)
            print("\b\b "+self.sense[i]+"= "+str(self.b[i]))
        return

    def print_slack(self)->None:
        '''
            Print Slacks for each constraints
        '''
        print("\nConstraint values at Optimal solution:\n",self.constraint@self.optimal_x)
        print("Binding values:", self.b)
        print("Slack: ", abs((self.constraint@self.optimal_x)-self.b))
        return

    def sensitivity_analysis(self, verbose = False)->None:
        '''
            Sensitivity Analysis
        '''
        shadow_cost = {index: con.Pi for index, con in enumerate(self.model_constraints)}
        shadow_cost_bound = {index: (con.SARHSLow, con.SARHSUp) for index, con in enumerate(self.model_constraints)}
        if verbose:
            try:
                print("\nShadow Cost:")
                # shadow_cost = [con.Pi for con in self.model_constraints]
                print("\t", shadow_cost)
            except:
                print("\tCan not fetch value for Shadow Cost")

            try:
                print("\nUpper and Lower Bound of Shadow Cost")
                # shadow_cost_bound = [(con.SARHSLow, con.SARHSUp) for con in self.model_constraints]
                print("\t", shadow_cost_bound)
            except:
                print("\tCan not fetch value for Upper and Lower")

            try:
                print("\nRange of Objective where Optimal Corner remains the same")
                print("\tLower Bound:",self.model_X.SAObjLow)# <- Range of Slope for which optimal Corner remains same
                print("\tUpper Bound:",self.model_X.SAObjUP) # <- Range of Slope for which optimal Corner remains same
            except:
                print("\tCan not fetch value for Optimal Corner")        

        return {
            'shadow_cost': shadow_cost,
            'shadow_cost_bound': shadow_cost_bound,
            'optimal_corner_range': {"low": self.model_X.SAObjLow,
                                    'high': self.model_X.SAObjUP}
        }

# Question 1

Use 2019 stock data _(stocks2019.csv)_ to find a portfolio that maximizes expected return while maintaining that no more than 100 days have a negative return.  Short selling is not allowed.  Apply this portfolio to 2020 data _(stocks2020.csv)_, how many days in 2020 had negative returns?

 

To solve this problem set your objective to be the mean return of your portfolio.  Then define a binary variable for each days' return that takes on the value 1 if that day's return is negative, and 0 if it's positive (enforce this with a big-M constraint) and then add a constraint that the sum of these binary variables must be less than or equal to 100.  This is an integer programming problem.

In [26]:
stocks2019 = pd.read_csv('stocks2019.csv')
stocks2019.X = pd.to_datetime(stocks2019.X)
stocks2019.set_index('X', inplace= True)
#Changing price levels to returns and discarding first row
stocks2019 = stocks2019.pct_change().iloc[1:]
print(f'The Stocks Dataset has {stocks2019.shape[0]} days data for {stocks2019.shape[1] - 1} stocks following the "NDX" index')
stocks2019

The Stocks Dataset has 250 days data for 100 stocks following the "NDX" index


,NDX,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
X,,,,,,,,,,,,,,,,,,,,,
2019-01-03,-0.033602,-0.035509,-0.039498,-0.094530,0.022030,-0.085791,-0.027696,-0.028484,-0.025242,-0.015216,...,-0.022834,-0.018591,-0.034989,-0.030557,-0.002133,-0.010435,-0.035808,-0.068217,-0.003940,-0.037651
2019-01-04,0.044824,0.039903,0.048632,0.114370,0.057779,0.010445,0.051294,0.053786,0.050064,0.034184,...,0.058976,0.047954,0.044744,0.044147,0.054785,0.033269,0.060577,0.042356,0.009786,0.046303
2019-01-07,0.010211,0.028196,0.013573,0.082632,0.018302,0.017192,-0.001994,-0.002167,0.034353,0.013457,...,0.022067,0.062620,0.016312,0.001000,0.018240,0.005749,0.000918,0.035522,-0.004330,0.026415
2019-01-08,0.009802,0.030309,0.014918,0.008751,0.006207,0.015954,0.008783,0.007385,0.016612,0.012824,...,0.010281,0.018450,0.036460,0.008902,0.013307,0.020009,0.021101,-0.009615,0.011597,-0.019051
2019-01-09,0.007454,0.017210,0.011819,-0.026988,0.012430,0.038196,-0.003427,-0.001505,0.001714,-0.001196,...,0.023745,0.018804,-0.008157,0.003781,0.023065,0.012050,-0.004612,0.044083,-0.007984,0.008972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,0.002019,-0.005572,0.004090,0.029672,0.006469,0.019239,-0.000437,-0.000556,0.003638,-0.000123,...,0.002603,0.010060,-0.000104,-0.000869,-0.003217,0.003083,-0.009273,0.026019,-0.017615,0.011868
2019-12-24,0.000402,-0.001358,0.002098,0.023757,-0.001630,-0.000899,-0.004590,-0.003914,-0.002114,-0.002880,...,-0.005482,-0.002095,0.001663,-0.002541,0.002637,-0.003756,0.003502,-0.007496,0.003913,-0.004146
2019-12-26,0.009058,0.001360,0.004732,0.001934,-0.012242,0.001331,0.013418,0.012534,0.044467,-0.001774,...,0.002901,-0.004594,0.004981,-0.002279,-0.001406,0.009426,-0.004994,0.009160,-0.001114,0.001929


Objective: Maximise Portfolio Returns s.t. Returns can not be -ve for ≥ 100 days

Let portfolio $P$ be defined as:
$$P = w_0x_0 + 

# Question 2

You are the publisher for 5 local newspapers.  The previous 50 days worth of demand is included in the following file

_hw2_demand.csv_

It costs you $0.50, $0.50, $0.25, $0.75, $0.25  to print each copy of each of the 5 newspapers.

The newspapers sell for $1.25, $1.20, $0.50, $1.50, $0.60 each.

 

Solve the multi-product newsvendor problem.

 
How much profits do you expect to make tomorrow?  Round to the nearest cent.

# Question 3

Go back to the saving for retirement problem we did in class.  There's approximately a 55% chance you go bankrupt before you die.  That's not very good.  You should probably keep working for a few more years.  Today you have $2M in your retirement account.  If you keep working, you earn $400k (after taxes) per year, with a 2.5% raise every year.  You still spend $150k per year and when you work, every dollar of salary that you don't spend goes into your retirement account. As in the code from class, each year first grow your portfolio from market returns, then add your salary and give yourself a raise for next year (if you're still working), then subtract your cost of living. You're still going to die 35 years from today.  The market has normal annual returns with mean = 7% and sd = 10%.  What is the smallest number of years you need to work such that your probability of going bankrupt before death is less than 6%?  Use 10000 simulations.

# Question 4

You manufacture microprocessors.  There are 2 lines of chips: high performance and low performance.  Your manufacturing processes are not perfect though, there are quality issues in your production. There are 2 manufacturing processes you can perform.  One process produces chips where the performance is a normal random variable with mean = 10 and sd = 3, and it costs $10 to manufacture chips using this process.  The other manufacturing process produces chips where the performance is a normal random variable with mean = 5 and sd = 2, and it costs $5 to manufacture chips using this process.  Any chip that has a performance less than 7 must be sold as a low performance chip, and any chip that has a performance above 7 can be sold as a high performance chip.  Low performance chips sell for $8 and high performance chips sell for $15.  There is currently a global chip shortage, so any chip you manufacture will sell immediately.  If you produce 100 chips using the first process and 200 chips using the second process, what is the standard deviation of your profit?  Use 10000 simulations to answer this question.